In [37]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [15]:
import time
import warnings

warnings.filterwarnings(action='ignore')

In [47]:
urls = 'https://map.kakao.com/'
item = '맛집'

In [46]:
driver = webdriver.Chrome('../webdriver/chromedriver')

In [48]:
driver.get(urls) # 카카오 지도 접속하기

In [49]:
searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기 
searchbox.send_keys(item)

In [50]:
searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
driver.execute_script("arguments[0].click();", searchbutton)

In [54]:
if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                try:
                    img_url = review_lists[i].select_one('a.link_photo > img ')['src']
                except:
                    continue
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"UserID":None,"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(3)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        try:
                            img_url = review_lists[i].select_one('a.link_photo > img ')['src']
                        except:
                            continue
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"UserID":user_id[0].get('data-userid'), "ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try:
                                img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except:
                                img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        print("식당 존재 x")
        
else:
    pass

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 124)

In [ ]:
rating_list = []
userid_list = []
itemid_list = []
timestamp_list = []
comment_list = []

for k, row in nowon.iterrows():
    url = nowon["kakao_url"][k]
    print(k,"번째 크롤링 중 : ",nowon["상호명"][k] , sep=" ")
    driver.get(url)

    time.sleep(2)
    
    try:
        rating_num = driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_sorting > a > span.color_b').text
        rating_num = int(rating_num)
        print(rating_num)
        
        # 별점 개수로 페이지를 넘기는 기준을 정하기로 함 / 한 페이지 당 최대 리뷰는 5개!
        if rating_num > 0 and rating_num <= 5:
            p_num = 1
            print(p_num)
            
            time.sleep(2)
           
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            user = driver.find_elements_by_xpath("//a[@class='link_user']")
            review_lists = soup.select('.list_evaluation > li')
            time.sleep(3)
            for j in range(0, len(user)):
                userid_list.append(user[j].get_attribute("data-userid")) # 새로 생성한 userid_list에 userid를 추출해 더합니다.
                itemid_list.append(nowon["상호명"][k])
                timestamp_list.append(soup.select('.time_write')[j].text)
                time.sleep(3)
            if len(review_lists) != 0:
                for j, review in enumerate(review_lists):
                    comment = review.select('.txt_comment > span') 
                    rating = review.select('.grade_star > em') # 별점
                            
                    if len(comment) != 0:
                        if len(rating) != 0 :
                            comment_list.append(comment[0].text) 
                            rating_list.append(rating[0].text)
                            time.sleep(3)
                        else:
                            comment_list.append(comment[0].text) 
                            rating_list.append('0')
            else:
                print('리뷰가 없습니다.')
                time.sleep(3)    
                    
                    
        # 2 페이지
        elif rating_num >= 6 and rating_num <= 10:
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            user = driver.find_elements_by_xpath("//a[@class='link_user']")
            review_lists = soup.select('.list_evaluation > li')
            time.sleep(3)
            for j in range(0, len(user)):
                userid_list.append(user[j].get_attribute("data-userid")) # 새로 생성한 userid_list에 userid를 추출해 더합니다.
                itemid_list.append(nowon["상호명"][k])
                timestamp_list.append(soup.select('.time_write')[j].text)
                time.sleep(3)
            if len(review_lists) != 0:
                for j, review in enumerate(review_lists):
                    comment = review.select('.txt_comment > span') 
                    rating = review.select('.grade_star > em') # 별점
                            
                    if len(comment) != 0:
                        if len(rating) != 0:
                            comment_list.append(comment[0].text) 
                            rating_list.append(rating[0].text)
                            time.sleep(3)
                        else:
                            comment_list.append(comment[0].text) 
                            rating_list.append('0')
            else:
                print('리뷰가 없습니다.')
                time.sleep(3)
                    
            driver.find_element_by_xpath('//*[@id="mArticle"]/div[4]/div[4]/div/a').click()
            time.sleep(2)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            user = driver.find_elements_by_xpath("//a[@class='link_user']")
            review_lists = soup.select('.list_evaluation > li')
            time.sleep(3)
            
            for j in range(0, len(user)):
                userid_list.append(user[j].get_attribute("data-userid")) # 새로 생성한 userid_list에 userid를 추출해 더합니다.
                itemid_list.append(nowon["상호명"][k])
                timestamp_list.append(soup.select('.time_write')[j].text)
                time.sleep(3)
            if len(review_lists) != 0:
                for j, review in enumerate(review_lists):
                    comment = review.select('.txt_comment > span') 
                    rating = review.select('.grade_star > em') # 별점
                            
                    if len(comment) != 0:
                        if len(rating) != 0:
                            comment_list.append(comment[0].text) 
                            rating_list.append(rating[0].text)
                            time.sleep(3)
                        else:
                            comment_list.append(comment[0].text) 
                            rating_list.append('0')
            else:
                print('리뷰가 없습니다.')
                time.sleep(3)
                    
        # 3페이지 이하         
        elif rating_num >= 11 and rating_num <= 15:
            p_num = 3
            print(p_num)
                 
            for i in range(1,p_num+1):
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                user = driver.find_elements_by_xpath("//a[@class='link_user']")
                review_lists = soup.select('.list_evaluation > li')
                time.sleep(3)
                for j in range(0, len(user)):
                    userid_list.append(user[j].get_attribute("data-userid")) # 새로 생성한 userid_list에 userid를 추출해 더합니다.
                    itemid_list.append(nowon["상호명"][k])
                    timestamp_list.append(soup.select('.time_write')[j].text)
               
            # 카카오맵의 개인 user별 url의 규칙은 다음과 같습니다.
                    time.sleep(3)
                if len(review_lists) != 0:
                    for j, review in enumerate(review_lists):
                        comment = review.select('.txt_comment > span') 
                        rating = review.selec('.grade_star > em')
                            
                        if len(comment) != 0:
                            if len(rating) != 0:
                                comment_list.append(comment[0].text) 
                                rating_list.append(rating[0].text)
                                time.sleep(3)
                            else:
                                comment_list.append(comment[0].text) 
                                rating_list.append('0')
                else:
                    print('리뷰가 없습니다.')
                try: # 마지막 페이지 크롤링을 위해서 try사용
                    driver.find_element_by_xpath('//*[@id="mArticle"]/div[4]/div[4]/div/a['+ str(i) +']').click() # 페이지별로 실행하고
                except:
                    continue
                time.sleep(2)
               
                
        # 4페이지     
        elif rating_num >= 16 and rating_num <= 20:
            p_num = 4
            print(p_num)
            
            for i in range(1,p_num+1):
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                user = driver.find_elements_by_xpath("//a[@class='link_user']")
                review_lists = soup.select('.list_evaluation > li')
                time.sleep(2)
                for j in range(0, len(user)):
                    userid_list.append(user[j].get_attribute("data-userid")) # 새로 생성한 userid_list에 userid를 추출해 더합니다.
                    itemid_list.append(nowon["상호명"][k])
                    timestamp_list.append(soup.select('.time_write')[j].text)
            # 카카오맵의 개인 user별 url의 규칙은 다음과 같습니다.
                time.sleep(2)
               
                if len(review_lists) != 0:
                    for j, review in enumerate(review_lists):
                        comment = review.select('.txt_comment > span') 
                        rating = review.select('.grade_star > em')
                            
                        if len(comment) != 0:
                            if len(rating) != 0:
                                comment_list.append(comment[0].text) 
                                rating_list.append(rating[0].text)
                                time.sleep(3)
                            else:
                                comment_list.append(comment[0].text) 
                                rating_list.append('0')
                else:
                    print('리뷰가 없습니다.')
                try: # 마지막 페이지 크롤링을 위해서 try사용
                    driver.find_element_by_xpath('//*[@id="mArticle"]/div[4]/div[4]/div/a['+ str(i) +']').click() # 페이지별로 실행하고
                except:
                    continue
                time.sleep(2)
            
                        
            
                        
        # 5페이지 이상           
        elif rating_num >= 21:
            p_num = int(math.ceil(rating_num / 5))
            print(p_num)
            
            for i in range(1, 7):
                p_num -= 1
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                user = driver.find_elements_by_xpath("//a[@class='link_user']")
                review_lists = soup.select('.list_evaluation > li')
                time.sleep(2)
                for j in range(0, len(user)):
                    userid_list.append(user[j].get_attribute("data-userid")) # 새로 생성한 userid_list에 userid를 추출해 더합니다.
                    itemid_list.append(nowon["상호명"][k])
                    timestamp_list.append(soup.select('.time_write')[j].text)
             # 카카오맵의 개인 user별 url의 규칙은 다음과 같습니다.
                    time.sleep(3)
              
                
                    # 리뷰 리스트가 0이 아니면
                if len(review_lists) != 0:
                    for j, review in enumerate(review_lists):
                        comment = review.select('.txt_comment > span') 
                        rating = review.select('.grade_star > em') # 별점
                            
                        if len(comment) != 0:
                            if len(rating) != 0:
                                comment_list.append(comment[0].text) 
                                rating_list.append(rating[0].text)
                                time.sleep(3)
                            else:
                                comment_list.append(comment[0].text) 
                                rating_list.append('0')
                else:
                    print('리뷰가 없습니다.')
                try: # 마지막 페이지 크롤링을 위해서 try사용
                    driver.find_element_by_xpath('//*[@id="mArticle"]/div[4]/div[4]/div/a['+ str(i) +']').click() # 페이지별로 실행하고
                except:
                    continue
                time.sleep(2)
                print(p_num)
                pp_num = p_num
            while p_num <= pp_num and p_num >= 0:
                for a in range(2,7) :
                    p_num -= 1
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    review_lists = soup.select('.list_evaluation > li')
                    user = driver.find_elements_by_xpath("//a[@class='link_user']")
                    time.sleep(2)
                    for j in range(0,len(user)):
                        userid_list.append(user[j].get_attribute("data-userid")) # 새로 생성한 userid_list에 userid를 추출해 더합니다.
                        itemid_list.append(nowon["상호명"][k])
                        timestamp_list.append(soup.select('.time_write')[j].text)
                        time.sleep(2)
                    if len(review_lists) != 0:
                        for i, review in enumerate(review_lists):
                            comment = review.select('.txt_comment > span') # 리뷰
                            rating = review.select('.grade_star > em') # 별점
                            if len(comment) != 0:
                                if len(rating) != 0:
                                    comment_list.append(comment[0].text) 
                                    rating_list.append(rating[0].text)
                                    time.sleep(3)
                                else:
                                    comment_list.append(comment[0].text) 
                                    rating_list.append('0')
                    else:
                        print('no review in extract')
                    try:
                        driver.find_element_by_xpath('//*[@id="mArticle"]/div[4]/div[4]/div/a['+ str(a) +']').click() # 페이지별로 실행하고
                    except:
                        continue
                    time.sleep(3)    
              
            
                            
                                       

    except (NoSuchElementException, ElementNotInteractableException):
        print("리뷰가 없습니다")
        continue

In [ ]:
count = 0
current = 0
goal = len(items)

for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                try:
                    img_url = review_lists[i].select_one('a.link_photo > img ')['src']
                except:
                    continue
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"UserID":None,"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(3)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        try:
                            img_url = review_lists[i].select_one('a.link_photo > img ')['src']
                        except:
                            continue
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"UserID":user_id[0].get('data-userid'), "ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try:
                                img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except:
                                img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"UserID":user_id[0].get('data-userid'),"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        print("식당 존재 x")

In [13]:
driver.close()